![NeuronUnit Logo](https://raw.githubusercontent.com/scidash/assets/master/logos/neuronunit-logo-text.png)
# Chapter 3
Back to [Chapter 2](chapter2.ipynb)

In the second chapter we tested a real ion channel model using data from the [OpenWorm](http://openworm.org) project.  Here we'll test a reduced neuron model using data from the [NeuroElectro](http://neuroelectro.org) project and from the [Allen Brain Institute Cell Types](http://celltypes.brain-map.org) database.  
### We'll test an Izhikevich model against data from a Layer V pyramidal cell in primary visual cortex

In [1]:
%matplotlib inline
import os,sys
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
import sciunit
import neuronunit
from neuronunit import aibs
from neuronunit.models.reduced import ReducedModel

In [2]:
import quantities as pq
from neuronunit import tests as nu_tests, neuroelectro
neuron = {'nlex_id': 'nifext_50'} # Layer V pyramidal cell
tests = []

dataset_id = 354190013  # Internal ID that AIBS uses for a particular Scnn1a-Tg2-Cre 
                        # Primary visual area, layer 5 neuron.
    
# Obtain the empirical rheobase current from the Allen Brain Insitute Cell Types database.  
observation = aibs.get_observation(dataset_id,'rheobase')
rheobase_test = nu_tests.RheobaseTest(observation=observation)
tests += [rheobase_test]
    
test_class_params = [(nu_tests.InputResistanceTest,None),
                     (nu_tests.TimeConstantTest,None),
                     (nu_tests.CapacitanceTest,None),
                     (nu_tests.RestingPotentialTest,None),
                     (nu_tests.InjectedCurrentAPWidthTest,None),
                     (nu_tests.InjectedCurrentAPAmplitudeTest,None),
                     (nu_tests.InjectedCurrentAPThresholdTest,None)
                    ]

# Obtain all other parameters from neuroelectro.org.
for cls,params in test_class_params:
    observation = cls.neuroelectro_summary_observation(neuron)
    tests += [cls(observation,params=params)]
    
# A hook to update all tests after the RheobaseTest to use the rheobase current (to produce exactly one AP)
def update_amplitude(test,tests,score):
    rheobase = score.prediction['value']
    for test in tests[1:]:
        if 'Injected' in test.name:
            # Set current injection to just suprathreshold
            test.params['injected_square_current']['amplitude'] = rheobase*1.01 
    
hooks = {tests[0]:{'f':update_amplitude}}
suite = sciunit.TestSuite(tests,name="vm_suite",hooks=hooks)

Getting Rheobase cached data value for from AIBS dataset 354190013
Getting Input Resistance data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__name=Input+Resistance&nlex=nifext_50
Getting Membrane Time Constant data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__name=Membrane+Time+Constant&nlex=nifext_50
Getting Cell Capacitance data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__name=Cell+Capacitance&nlex=nifext_50
Getting Resting membrane potential data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__name=Resting+membrane+potential&nlex=nifext_50
Getting Spike Half-Width data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__name=Spike+Half-Width&nlex=nifext_50
Getting Spike Amplitude data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__name=Spike+Amplitude&nlex=nifext_50
Getting Spike Threshold data values from neuroelectro.org
http://neuroelectro.org/api/1/nes/?e__nam

### The Izhikevich model here is defined in an NeuroML2/LEMS file.  
### We will run it using the jNeuroML backend for simplicity, although this is *much* slower than the native NEURON backend.  

In [3]:
# This example is from https://github.com/OpenSourceBrain/IzhikevichModel.
for rel_path in ['.','docs','../docs']:
    # Check to see if this is the neuronunit/docs directory.
    DOCS_PATH = os.path.abspath(os.path.join(os.getcwd(),rel_path)) 
    if DOCS_PATH.endswith('neuronunit/docs'):
        break
assert DOCS_PATH.endswith('neuronunit/docs'), ("Could not find the path to neuronunit/docs. "
                                               "Change the DOCS_PATH variable to the path to "
                                               "your neuronunit/docs directory.")
LEMS_MODEL_PATH = os.path.join(DOCS_PATH,'data/NeuroML2/LEMS_2007One.xml')
model = ReducedModel(LEMS_MODEL_PATH,name='vanilla',backend='jNeuroML')

### Run a series of tests.  The RheobaseTest is run first to obtain the rheobase current to use in all the other tests.

In [4]:
score_matrix = suite.judge(model)

pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmp46doda0y, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs
Injected 300.0 pA current and got 17 spikes
pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmp0bu0m549, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs
Injected -0.0 pA current and got 0 spikes
pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg

pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpi192r5ow, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs
pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpjwi8z4by, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs


pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmprc9xx5ja, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs


pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpa_c2e9xh/vanilla.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpu_mgo1lc, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs


### Now we can sweep across a parameter (the resting potential) and run the same test suite on a model corresponding to each value of this parameter.

In [5]:
models = []
for vr in np.linspace(-80,-50,4):
    model = ReducedModel(LEMS_MODEL_PATH, 
                         name='V_rest=%dmV' % vr, 
                         attrs={'//izhikevich2007Cell':
                                    {'vr':'%d mV' % vr}
                               })
    #model.skip_run = True
    models.append(model)
suite.set_verbose(False) # Turn off most print statements.  

In [6]:
score_matrix = suite.judge(models[0])

pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpp_fgr6s3, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs
pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpp871lxef, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs
pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmphocuyv2n, cwd: /Users/

pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmp3dz6u66j, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs


pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpcfwsfqbu, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs


pyNeuroML >>> Reloading data specified in LEMS file: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml (/private/var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmpt7q8xdbr/V_rest=-80mV.xml), base_dir: /var/folders/_j/vg2m860n23d_9ty1h2z9_2880000gn/T/tmp8xwvdnur, cwd: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/docs


In [7]:
score_matrix.show_mean = True # Show the mean value across test scores. 
                              # The mean is computed using the sort_key attribute, which is in the range [0,1] for
                              # all Score types
score_matrix.sortable = True # Make the ScoreMatrix sortable (and searchable)
score_matrix

<IPython.core.display.Javascript object>

,Mean,RheobaseTest,InputResistanceTest,TimeConstantTest,CapacitanceTest,RestingPotentialTest,InjectedCurrentAPWidthTest,InjectedCurrentAPAmplitudeTest,InjectedCurrentAPThresholdTest
V_rest=-80mV,0.323,Ratio = 1.89,Z = -1.04,Z = -1.74,Z = -0.54,Z = -1.80,Z = -0.80,Z = -1.22,Z = 1.59


### Let's take a look at a sweep from one of these models ($V_{rest} = -55 mV$) obtained at the rheobase current.  

In [8]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 18
score_matrix['RheobaseTest']['V_rest=-60mV'].plot_vm() # Plot the rheobase current from the model with V_rest = -60 mV

KeyError: "No model or test with name 'None'"